In [1]:
from owlready2 import Thing, get_ontology, AllDisjoint, sync_reasoner_pellet, Imp, FunctionalProperty
from owlready2 import *

# Create a new ontology
onto = get_ontology("http://example.org/onto#")

# Define classes for the ontology based on the CSV file contents
with onto:
    class TVShow(Thing): pass
    class Episode(Thing): pass
    class User(Thing): pass
    class Comment(Thing): pass
    class Rating(Thing): pass
    class Emotion(Thing): pass

    # Define properties
    class aPourTitre(TVShow >> str, FunctionalProperty): pass

    class aPourTVShow(Episode >> TVShow, FunctionalProperty): pass
    class aPourSaison(Episode >> int, FunctionalProperty): pass
    class aPourNumero(Episode >> int, FunctionalProperty): pass
    class rewatchedCount(Episode >> int, FunctionalProperty): pass
    # class aPourDate(Episode >> str, FunctionalProperty): pass
    class aPourAnnee(Episode >> int, FunctionalProperty): pass
    class aPourMois(Episode >> int, FunctionalProperty): pass

    class aPourNote(Rating >> int, FunctionalProperty): pass
    # class aPourDate(Rating >> str, FunctionalProperty): pass

    class aPourContenu(Comment >> str, FunctionalProperty): pass
    # class aPourDate(Comment >> str, FunctionalProperty): pass
    
    class aPourEmotion(Emotion >> str, FunctionalProperty): pass
    # class aPourDate(Emotion >> str, FunctionalProperty): pass
    
    # Define relationships
    class posts_by(Comment >> User): pass

    class comments_on(Comment >> Episode): pass
    class follows(User >> TVShow): pass
    class has_started(User >> TVShow): pass
    class had_stopped(User >> TVShow): pass
    class has_watched(User >> Episode): pass

    class rated_by(Rating >> User): pass
    class rates(Rating >> Episode): pass

    class expressed_by(Emotion >> User): pass
    class evokes(Emotion >> Episode):  pass



* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
import csv

def load_data(filename, process_row):
    with open(filename, 'rt', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=',')
        count = 0
        for row in reader:
            if count > 0 and count%100000 == 0:
                print(f'Processed {count} rows')
            process_row(row)
            count += 1

In [3]:
import datetime

def process_tv_shows_episodes(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])
    tv_show.aPourTitre = row['tv_show_name']

    episode = onto.Episode(name=row['episode_id'])
    episode.aPourTVShow = tv_show
    episode.aPourSaison = int(row['episode_season_number'])
    episode.aPourNumero = int(row['episode_number'])
    episode.rewatchedCount = int(row['rewatched_count'])
    # episode.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error
    date = datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S')
    annee = date.year
    episode.aPourAnnee = int(annee)
    mois = date.month
    episode.aPourMois = int(mois)

    user = onto.User(name=row['user_id'])
    user.has_started.append(tv_show) # user has started watching the tv show because they have seen at least one episode
    user.has_watched.append(episode) # user has watched the episode

path = './data/seen_episode_modified.csv'
load_data(path, process_tv_shows_episodes)
print(f'Loaded {len(onto.TVShow.instances())} TV shows and {len(onto.Episode.instances())} episodes seen and {len(onto.User.instances())} users')

Loaded 106 TV shows and 5227 episodes seen and 1 users


In [4]:
def process_episodes_ratings(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    rating = onto.Rating(name=row['vote_key'])
    rating.aPourNote = int(row['note'])
    # rating.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error

    rating.rated_by.append(user)
    rating.rates.append(episode)

path = './data/ratings_episode_votes_modified.csv'
load_data(path, process_episodes_ratings)
print(f'Loaded {len(onto.Rating.instances())} ratings')

Loaded 339 ratings


In [5]:
def process_tv_shows_followings(row):
    tv_show = onto.TVShow(name=row['tv_show_name'])

    user = onto.User(name=row['user_id'])
    user.follows.append(tv_show)
    if row['active'] == '0' or row['archived'] == '1': # user has stopped following the tv show
        user.had_stopped.append(tv_show)

path = './data/followed_tv_show_modified.csv'
load_data(path, process_tv_shows_followings)
print(f'Loaded {len(onto.User.instances())} users and {len(onto.TVShow.instances())} TV shows')

Loaded 1 users and 119 TV shows


In [6]:
def process_episodes_comments(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    comment = onto.Comment(name=row['comment_id'])
    comment.aPourContenu = row['comment']
    # comment.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date()) # cast datetime to string to avoid error

    comment.posts_by.append(user)
    comment.comments_on.append(episode)

path = './data/episode_comment_modified.csv'
load_data(path, process_episodes_comments)
print(f'Loaded {len(onto.Comment.instances())} comments')

Loaded 35 comments


In [7]:
def process_episodes_emotions(row):
    episode = onto.Episode(name=row['episode_id'])
    user = onto.User(name=row['user_id'])
    emotion = onto.Emotion(name=row['emotion_vote_id'])
    emotion.aPourEmotion = row['emotion_id']
    # emotion.aPourDate = str(datetime.datetime.strptime(row['created_at'], '%Y-%m-%d %H:%M:%S').date())
    emotion.expressed_by.append(user)
    emotion.evokes.append(episode)

path = './data/emotions_episode_votes_modified.csv'
load_data(path, process_episodes_emotions)
print(f'Loaded {len(onto.Emotion.instances())} emotions')

Loaded 341 emotions


In [8]:
sync_reasoner_pellet([onto], infer_property_values = True, infer_data_property_values = True, debug=True)

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\antlr-runtime-3.2.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\aterm-java-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\commons-codec-1.6.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\owlready2\pellet\httpclient-4.2.3.jar;C:\Users\romai\AppData\Local\Packages\PythonSoftwareFoundation.Pyth

* Owlready * Adding relation onto.34759499 aPourContenu Damn, Stephan breaking the circle of violence was breathtaking\°46 respect bruv
* Owlready * Adding relation onto.32426747 aPourContenu So sad we don\°31t see Kim and Nicholas sitting together in the bar and talking as friends \?75\?42 I would have shed a tear
* Owlready * Adding relation onto.33337268 aPourContenu I\°31m starting to get bored but what a plot it would be to see zacarias destroying the trafiquants because they try to black mail him
* Owlready * Adding relation onto.32406536 aPourContenu Damn who is spying who it\°31s more complicated that I thought
* Owlready * Adding relation onto.32606931 aPourContenu I love so much the mentality : they are all awesome but there also all fans of each other and they respect everyone some much I love this sport\°31s mentality !
* Owlready * Adding relation onto.33012136 aPourContenu This guy is like the embodiment of wisdom \?75\?55
* Owlready * Adding relation onto.31913600 aPourC

* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


In [9]:
onto.save(file = "tvtime_ontology.owl", format = "rdfxml")